In [1]:
from collections import defaultdict
import json
import random
import secrets

In [4]:
# Simulation constants
CATEGORIES = ['eth_core', 'op_tooling', 'op_rnd']
NUM_CATEGORIES = len(CATEGORIES)
NUM_PROJECTS = 100
NUM_VOTERS = 100
MAX_OP = 1_000_000
MIN_OP = 1000
TOTAL_OP = 8_000_000
MAX_COI_PROJECTS = 10

In [5]:
# Initialize categories and projects

# Randomly generate project UIDs
projects = [
    f"0x{secrets.token_hex(32)}"
    for _ in range(NUM_PROJECTS)
]

# Randomly allocate projects to categories
projects_by_category = defaultdict(list)
for project in projects:
    category = random.choice(CATEGORIES)
    projects_by_category[category].append(project)

# Test
projects_by_category['op_tooling']

['0x61ba972590a5d0a76a1a948dc8c328c2a6242aec822018200788a7c5babaabac',
 '0x09c8a4b5407c62b342aa3737d2b0a45364d6fa7867d028ba5169221e6fd694bc',
 '0x7e1ee810b60eec3b703e6db02550401f915049d2d02f557e2dd41c93708f5c21',
 '0x859466ef73b6991aebad89f9a3b6ec56581ebcd7a45d8270ee36aa39bd21a4e5',
 '0xdbfe0ee23e645e4d7323cae80919a23cabfa87ce09ca3986c6efb3422ea90db5',
 '0xe63c69bf5b748f5660107720e5abbc20b3b499134933d6faf466d5d876957797',
 '0x0e86fdd04b544d2e6214f44bab5e0cb5b25a459db8813c2ca5b576885a3c8e97',
 '0x0612554881ba9aa5bffc9cda7ec3a414995afb1cceb705adb0730c6bf33c1762',
 '0xaf2782ad1e11e6ee32f93b326f5961ec8c8752834f0a1b5a1d31c1b2752b1941',
 '0x2812a63b8a270f80f4e9ca47b06f7d6872f5e2bbfa474e082129f116dbbe73e6',
 '0x7cbc2a042b24ab34f2cf767b2c635ebd63b4d090a16333ea1a7f308b4e5cbfbe',
 '0x5693db309b2e3db862bd8baa40a9e76a0240f12ccd746ad6b55ac4cd5be21d34',
 '0xc5c5c6b6a7c68fa442b2085ad897069cdf2c3d9d711077b76ecc9bb422896985',
 '0x021e6e1ff49e11bf4e0976501ccc12c542ae369eec772192388be9cedd6b0cab',
 '0xc4

In [6]:
# Generate voter ballots
def generate_ballot():

    # Generate random category for the voter
    assigned_category = random.sample(CATEGORIES, 1)[0]

    # Generate random COI list for each voter (for simplicity, random projects)
    coi_projects = list(random.sample(projects, random.randint(0, MAX_COI_PROJECTS)))

    # Create a random threshold (up to 10%) for the voter to "zero" vote projects they don't like
    zero_thresh = random.randint(0, 1000)/100 
    
    #  Determine category allocations
    category_allocations = {category: random.randint(0, 10000)/100. for category in CATEGORIES}
    total = sum(category_allocations.values())
    category_allocations = {k: (v * 100 / total) for k, v in category_allocations.items()}

    # Determine project allocations within a category
    for category, proj_list in projects_by_category.items():
        if category != assigned_category:
            continue

        project_allocations = defaultdict(list)
        for project in proj_list:            
            alloc = random.randint(0, 10000)/100.
            if alloc < zero_thresh or project in coi_projects:
                alloc = 0
            project_allocations[project] = alloc
        
        total = sum(project_allocations.values())
        project_allocations = {
            k: v * 100 / total if k not in coi_projects else None
            for k, v in project_allocations.items()
        }

    return {
        "voter_id": f"0x{secrets.token_hex(20)}",
        "category_allocations": category_allocations,
        "project_allocations": project_allocations
    }

# Generate all voter ballots
voter_ballots = [generate_ballot() for _ in range(NUM_VOTERS)]

In [7]:
voter_ballots[-1]

{'voter_id': '0xa44612da01605231677e4dbb21c464f97c047f9b',
 'category_allocations': {'eth_core': 21.383616383616385,
  'op_tooling': 42.67732267732268,
  'op_rnd': 35.939060939060944},
 'project_allocations': {'0x3fb02be6887e54023a5ef466a87bf2699bb51915388946bad061918ddc7372ff': 1.314087012776969,
  '0xde4a16f5e53fa4828545ff795d2f3a52cc91b54f7db3eab33d4e8de959ac857b': 8.870087336244541,
  '0x720d996d227d4829d008986f3b200562854e2858ee6ad6873d5ea983f14bae71': 7.697517386382016,
  '0xb21fb67655a28de6ce101bfa7380d7d02f0f71c70bd87a19a1c22529e45c3a9c': None,
  '0x6d905c4732a686a1f6860c1e64d8f0704d3213ec6934cd61c835b6111f020fc1': 5.634400776322173,
  '0xca5335357470b0bee691a32c4ae5645a85e575cec328d7ba278afe9f387cc42c': 7.2982371017305505,
  '0x3542e38f31412fccba991e3b893cffdff69bf10b3586e1efb4df0a05e0ed95a1': 3.657205240174672,
  '0xca4a8fd9c4585d5773ca36a07054939c0be7a5d9c0a2067cd2264d86b3cc1615': None,
  '0x9607cf6387e290e9a8d0def71cc4e147d334eea1e7c263ebd0e3c0e059b4c4f4': 2.702975901665858

In [8]:
with open("rf5_synthetic_ballots.json", "w") as f:
    json.dump(voter_ballots, f, indent=2)